In [48]:
# Imports e instalações

import pandas as pd
import numpy as np

# Viz
import matplotlib.pyplot as plt
import seaborn as sns

#!pip install scikit-learn
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Métricas
from sklearn.metrics import (accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix, classification_report)


In [49]:
df_clima = pd.read_csv(r'C:\Users\JacyzinGuilherme(Bip\mentoria-bip\dados_editados\australia_clima_v4.csv', sep=',')
df_clima = df_clima.sort_values("data").reset_index(drop=True)
df_clima

,data,localidade,temp_min,temp_max,chuva_mm,evaporacao,horas_sol,vento_rajada_direcao,vento_rajada_velocidade,vento_direcao_9h,...,ano,mes,dia,trimestre_quarter,horas_sol_isna,evaporacao_isna,nuvem_9h_isna,nuvem_15h_isna,pressao_9h_isna,pressao_15h_isna
0,2007-11-01,Canberra,8.0,24.3,0.0,3.4,6.3,NW,30.0,SW,...,2007,11,1,4,False,False,False,False,False,False
1,2007-11-02,Canberra,14.0,26.9,3.6,4.4,9.7,ENE,39.0,E,...,2007,11,2,4,False,False,False,False,False,False
2,2007-11-03,Canberra,13.7,23.4,3.6,5.8,3.3,NW,85.0,N,...,2007,11,3,4,False,False,False,False,False,False
3,2007-11-04,Canberra,13.3,15.5,39.8,7.2,9.1,NW,54.0,WNW,...,2007,11,4,4,False,False,False,False,False,False
4,2007-11-05,Canberra,7.6,16.1,2.8,5.6,10.6,SSE,50.0,SSE,...,2007,11,5,4,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56565,2017-06-25,Perth,6.3,17.0,0.0,1.6,7.9,E,26.0,SE,...,2017,6,25,2,False,False,False,False,False,False
56566,2017-06-25,Watsonia,7.9,13.0,0.0,2.8,3.8,NNW,39.0,N,...,2017,6,25,2,False,False,False,False,False,False
56567,2017-06-25,Sydney,7.6,19.3,0.0,3.4,9.4,W,35.0,W,...,2017,6,25,2,False,False,False,False,False,False
56568,2017-06-25,PerthAirport,4.9,16.8,0.0,1.6,7.9,E,39.0,E,...,2017,6,25,2,False,False,False,False,False,False


In [50]:
# Estatísticas Gerais
df_clima.info()
df_clima.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56570 entries, 0 to 56569
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   data                     56570 non-null  object 
 1   localidade               56570 non-null  object 
 2   temp_min                 56570 non-null  float64
 3   temp_max                 56570 non-null  float64
 4   chuva_mm                 56570 non-null  float64
 5   evaporacao               56570 non-null  float64
 6   horas_sol                56570 non-null  float64
 7   vento_rajada_direcao     56570 non-null  object 
 8   vento_rajada_velocidade  56570 non-null  float64
 9   vento_direcao_9h         56570 non-null  object 
 10  vento_direcao_15h        56570 non-null  object 
 11  vento_veloc_9h           56570 non-null  float64
 12  vento_veloc_15h          56570 non-null  float64
 13  umidade_9h               56570 non-null  float64
 14  umidade_15h           

,data,localidade,temp_min,temp_max,chuva_mm,evaporacao,horas_sol,vento_rajada_direcao,vento_rajada_velocidade,vento_direcao_9h,...,ano,mes,dia,trimestre_quarter,horas_sol_isna,evaporacao_isna,nuvem_9h_isna,nuvem_15h_isna,pressao_9h_isna,pressao_15h_isna
count,56570,56570,56570.000000,56570.000000,56570.000000,56570.000000,56570.000000,56570,56570.000000,56570,...,56570.000000,56570.000000,56570.000000,56570.000000,56570,56570,56570,56570,56570,56570
unique,3416,26,NaN,NaN,NaN,NaN,NaN,16,NaN,16,...,NaN,NaN,NaN,NaN,1,1,1,1,1,1
top,2010-06-09,Darwin,NaN,NaN,NaN,NaN,NaN,E,NaN,N,...,NaN,NaN,NaN,NaN,False,False,False,False,False,False
freq,25,3063,NaN,NaN,NaN,NaN,NaN,4531,NaN,4981,...,NaN,NaN,NaN,NaN,56570,56570,56570,56570,56570,56570
mean,NaN,NaN,13.458626,24.210032,2.131868,5.499295,7.732842,NaN,40.866343,NaN,...,2012.219445,6.423369,15.721584,2.477903,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,6.417872,6.973173,7.014007,3.696565,3.758660,NaN,13.340615,NaN,...,2.450910,3.450963,8.783534,1.121646,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,-6.700000,4.100000,0.000000,0.000000,0.000000,NaN,9.000000,NaN,...,2007.000000,1.000000,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,8.600000,18.700000,0.000000,2.800000,5.000000,NaN,31.000000,NaN,...,2010.000000,3.000000,8.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,13.200000,23.900000,0.000000,5.000000,8.600000,NaN,39.000000,NaN,...,2012.000000,6.000000,16.000000,2.000000,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,18.400000,29.700000,0.600000,7.400000,10.700000,NaN,48.000000,NaN,...,2014.000000,9.000000,23.000000,3.000000,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
df_clima.shape

(56570, 33)

In [52]:
# Valores Nulos
df_clima.isnull().sum()

data                       0
localidade                 0
temp_min                   0
temp_max                   0
chuva_mm                   0
evaporacao                 0
horas_sol                  0
vento_rajada_direcao       0
vento_rajada_velocidade    0
vento_direcao_9h           0
vento_direcao_15h          0
vento_veloc_9h             0
vento_veloc_15h            0
umidade_9h                 0
umidade_15h                0
pressao_9h                 0
pressao_15h                0
nuvem_9h                   0
nuvem_15h                  0
temp_9h                    0
temp_15h                   0
choveu_hoje_fex            0
chove_amanha_vtr           0
ano                        0
mes                        0
dia                        0
trimestre_quarter          0
horas_sol_isna             0
evaporacao_isna            0
nuvem_9h_isna              0
nuvem_15h_isna             0
pressao_9h_isna            0
pressao_15h_isna           0
dtype: int64

In [53]:
df_clima['chove_amanha_vtr'].value_counts(normalize=True)

chove_amanha_vtr
No     0.779477
Yes    0.220523
Name: proportion, dtype: float64

In [54]:
# Normalização da variável alvo
df_clima['chove_amanha_vtr'] = df_clima['chove_amanha_vtr'].map({'Yes': 1, 'No': 0})
df_clima['chove_amanha_vtr']

0        1
1        1
2        1
3        1
4        0
        ..
56565    0
56566    0
56567    0
56568    0
56569    0
Name: chove_amanha_vtr, Length: 56570, dtype: int64

In [55]:
df_clima['chove_amanha_vtr'].value_counts(normalize=True)

chove_amanha_vtr
0    0.779477
1    0.220523
Name: proportion, dtype: float64

### Regressão Logística

In [56]:
# Início do teste com modelo de Regressão Logística

# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

# Regressão logística é um modelo de classificação usado para prever a probabilidade de uma variável dependente categórica com base em uma ou mais variáveis independentes. É amplamente utilizado em problemas de classificação binária, onde a variável dependente tem dois resultados possíveis (por exemplo, sim/não, verdadeiro/falso).
# A regressão logística estima a probabilidade de um evento ocorrer, modelando a relação entre as variáveis independentes e a variável dependente usando a função logística (sigmóide). A saída do modelo é uma probabilidade que pode ser convertida em uma classe binária com base em um limiar predefinido (geralmente 0,5).
# Em regressão, ela observa todas as variáveis do dia, efetua um peso para cada uma delas, soma tudo e aplica uma função sigmóide para transformar o resultado em uma probabilidade entre 0 e 1.
# Se a probabilidade for maior que 0,5, o modelo prevê a classe "1" (evento ocorre); caso contrário, prevê a classe "0" (evento não ocorre).
# Exemplo:
# umidade = peso +2.0
# pressao = peso -1.5
# horas de sol = peso -3.0
# Todas essas variáveis são multiplicadas pelos seus respectivos pesos, somadas e aplicadas na função sigmóide para obter a probabilidade final // Chuva ou Não Chuva
# Probabilidade = sigmoid(w1*x1 + w2*x2 + ... + wn*xn + b)
# Onde: w1, w2, ..., wn são os pesos (coeficientes) das variáveis independentes
# x1, x2, ..., xn são as variáveis independentes
# b é o termo de interceptação (bias)
# Sigmoid entra a função matemática que transforma o resultado em uma probabilidade entre 0 e 1.
# O modelo passa os números pela função sigmóide para gerar uma probabilidade, que é então convertida em uma classe binária com base em um limiar predefinido (geralmente 0,5).
# A fórmula matemática da função sigmóide é:
# sigmoid(z) = 1 / (1 + e^(-z))
# Onde "e" é a base do logaritmo natural (aproximadamente 2,71828) e "z" é o valor resultante da soma ponderada das variáveis independentes mais o termo de interceptação.
# z (resultado da soma) // muito negativo // 0 // muito positivo || saída da sigmóide // 0 // 0,5 // 1
# O resultado é algo como 'probabilidade de chover amanhã é 0,8 (80%)', e se for maior que 0,5, o modelo prevê que vai chover (classe 1); caso contrário, prevê que não vai chover (classe 0).
# Esse modelo aprende os pesos (coeficientes) das variáveis independentes durante o treinamento, ajustando-os para minimizar a diferença entre as previsões do modelo e os resultados reais observados nos dados de treinamento.


In [57]:
# Definição de X e y
# y = resposta(chove amanhã)
# X = var preditora

X = df_clima.drop(columns=['chove_amanha_vtr'])
y = df_clima['chove_amanha_vtr']

X.shape, y.shape


((56570, 32), (56570,))

In [58]:
display(X.head(3)),display(y.head(3))

,data,localidade,temp_min,temp_max,chuva_mm,evaporacao,horas_sol,vento_rajada_direcao,vento_rajada_velocidade,vento_direcao_9h,...,ano,mes,dia,trimestre_quarter,horas_sol_isna,evaporacao_isna,nuvem_9h_isna,nuvem_15h_isna,pressao_9h_isna,pressao_15h_isna
0,2007-11-01,Canberra,8.0,24.3,0.0,3.4,6.3,NW,30.0,SW,...,2007,11,1,4,False,False,False,False,False,False
1,2007-11-02,Canberra,14.0,26.9,3.6,4.4,9.7,ENE,39.0,E,...,2007,11,2,4,False,False,False,False,False,False
2,2007-11-03,Canberra,13.7,23.4,3.6,5.8,3.3,NW,85.0,N,...,2007,11,3,4,False,False,False,False,False,False


0    1
1    1
2    1
Name: chove_amanha_vtr, dtype: int64

(None, None)

In [59]:
# Corte temporal dos dados para treino e teste

# split_point recebe o índice que separa os dados de treino (70%) dos dados de teste (30%)
split_point = int(len(X) * 0.7)

# X_train recebe os dados preditores de treino (70%)
# X_test recebe os dados preditores de teste (30%)
X_train = X.iloc[:split_point]
X_test  = X.iloc[split_point:]

# y_train recebe os dados alvo de treino (70%)
# y_test recebe os dados alvo de teste (30%)
y_train = y.iloc[:split_point]
y_test  = y.iloc[split_point:]

# Separação de tipo de variáveis (numéricas e categóricas)
# variaveis_numericas - armazene na var colunas com dados numéricos (int e float)
variaveis_numericas = X.select_dtypes(include=['int64', 'float64']).columns
# variaveis_categoricas - armazene na var colunas com dados categóricos (object e category)
variaveis_categoricas = X.select_dtypes(include=['object', 'category']).columns


In [60]:
# Pipeline numérico

# pipeline_numerico - recebe um pipeline para dados numéricos que primeiro preenche valores nulos com a mediana da coluna (SimpleImputer) e depois padroniza os dados (StandardScaler)
pipeline_numerico = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),('scaler', StandardScaler())])

# Pipeline categórico
# pipeline_categorico - recebe um pipeline para dados categóricos que primeiro preenche valores nulos com a moda da coluna (SimpleImputer) e depois aplica codificação one-hot (OneHotEncoder)
pipeline_categorico = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),('encoder', OneHotEncoder(handle_unknown='ignore'))])

# Pré processamento
# preprocessador - recebe um ColumnTransformer que aplique o pipeline_numerico às variáveis numéricas e o pipeline_categorico às variáveis categóricas
preprocessador = ColumnTransformer(transformers=[('num', pipeline_numerico, variaveis_numericas),('cat', pipeline_categorico, variaveis_categoricas)])

# Treino Reg Logística
# modelo_logistico - recebe um pipeline que primeiro aplique o preprocessador e depois treine um modelo de Regressão Logística (LogisticRegression) com max_iter=10000
modelo_logistico = Pipeline(steps=[('preprocessamento', preprocessador),('modelo', LogisticRegression(max_iter=10000))])
modelo_logistico.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessamento', ...), ('modelo', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transforme

In [61]:
# Prev
# variável y_pred recebe as previsões do modelo treinado
# modelo analisa X_test e decide para cadalinha se vai chover ou não
y_pred = modelo_logistico.predict(X_test)
# calcula acurácia comparando y_test (valores reais) com y_pred (valores previstos)
accuracy_score(y_test, y_pred)


0.8506864651464262

In [ ]:
# Matriz de confusão
# gera a matriz de confusão comparando y_test >>> valores reais e y_pred >>> valores previstos pelo modelo
# o resultado:::
# TN = acertos de "não chove"
# FP = falso alarme de chuva
# FN = disse "não chove", mas choveu
# TP = acertou a chuva
confusion_matrix(y_test, y_pred)

array([[12568,   756],
       [ 1778,  1869]])

In [63]:
"""
[[TN FP]
 [FN TP]]
TN = acertou “não chove”
FP = falso alarme de chuva
FN = disse “não chove”, mas choveu***
TP = acertou a chuva
"""

'\n[[TN FP]\n [FN TP]]\nTN = acertou “não chove”\nFP = falso alarme de chuva\nFN = disse “não chove”, mas choveu***\nTP = acertou a chuva\n'

In [ ]:
# var tscv recebe - cria um objeto TimeSeriesSplit que vai dividir os dados em 5 partes
# cada divisão respeita a ordem do tempo (passado >>> futuro)
tscv = TimeSeriesSplit(n_splits=5)

# cria uma lista vazia para armazenar a acurácia de cada divisão temporal
accuracies = []

# cria uma lista vazia para armazenar o recall da classe "chuva" (valor 1)
recalls_chuva = []

# para cada divisão temporal gerada pelo TimeSeriesSplit
# train_idx são os índices do conjunto de treino
# test_idx são os índices do conjunto de teste
for train_idx, test_idx in tscv.split(X):

    # seleciona os dados de treino usando os índices de treino
    X_train = X.iloc[train_idx]
    
    # seleciona os dados de teste usando os índices de teste
    X_test  = X.iloc[test_idx]

    # seleciona os valores reais de treino (chove ou não chove)
    y_train = y.iloc[train_idx]
    
    # seleciona os valores reais de teste
    y_test  = y.iloc[test_idx]

    # treina novamente o modelo logístico usando apenas os dados de treino
    # isso simula aprender com o passado
    modelo_logistico.fit(X_train, y_train)

    # usa o modelo treinado para prever os dados de teste
    # o resultado é uma lista de 0 e 1
    y_pred = modelo_logistico.predict(X_test)

    # calcula a acurácia dessa divisão temporal
    # compara o que o modelo previu com o que aconteceu de verdade
    accuracies.append(accuracy_score(y_test, y_pred))

    # calcula o recall da classe "chuva" (valor 1)
    # mede quantas chuvas reais o modelo conseguiu detectar
    recalls_chuva.append(recall_score(y_test, y_pred))
    
# calcula a média de todas as acurácias obtidas nas 5 divisões temporais
# isso mostra o desempenho médio do modelo ao longo do tempo
print(f"Acurácia: {np.mean(accuracies):.2f}")

# calcula a média dos recalls da classe "chuva"
# isso mostra, em média, quantas chuvas o modelo consegue identificar
print(f"Recall - chuva: {np.mean(recalls_chuva):.2f}")



Acurácia: 0.85
Recall - chuva: 0.55


### Random Forest

In [65]:
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
# https://scikit-learn.org/stable/modules/ensemble.html
# https://www.stat.berkeley.edu/~breiman/randomforest2001.pdf
# Random Forest é um algoritmo de aprendizado de máquina supervisionado usado para tarefas de classificação e regressão. Ele pertence à família dos métodos de ensemble, que combinam múltiplos modelos para melhorar a precisão e reduzir o risco de overfitting.
# O Random Forest constrói uma "floresta" de árvores de decisão durante o treinamento. Cada árvore é treinada em uma amostra aleatória dos dados de treinamento, e em cada nó da árvore, uma seleção aleatória de características é considerada para dividir os dados. Isso introduz diversidade entre as árvores, o que ajuda a melhorar a robustez do modelo.
# Para fazer previsões, o Random Forest agrega as previsões de todas as árvores na floresta. No caso de classificação, a classe final é determinada por votação majoritária entre as árvores, enquanto na regressão, a previsão final é a média das previsões das árvores individuais.
# O Random Forest é conhecido por sua capacidade de lidar com grandes conjuntos de dados, alta dimensionalidade, e por ser menos sensível a outliers em comparação com modelos individuais de árvore de decisão. Além disso, ele fornece medidas de importância das características, ajudando a identificar quais variáveis são mais relevantes para a previsão.    
# Ainda sobre:
# Afirma como funciona uma sequência de perguntas do tipo: "Se X for maior que Y, vá para a esquerda; se não, vá para a direita"
# Se umidade > 80?
# ├── Sim → Se pressão < 1010?
# │        ├── Sim → CHOVE
# │        └── Não → NÃO CHOVE
# └── Não → NÃO CHOVE
# Cada nó faz uma pergunta sobre uma característica específica dos dados, e a resposta a essa pergunta determina o próximo nó a ser visitado na árvore. Esse processo continua até que uma folha seja alcançada, onde uma decisão final é tomada (por exemplo, "CHOVE" ou "NÃO CHOVE").
# O Random Forest constrói várias dessas árvores de decisão, cada uma com suas próprias perguntas e decisões, e combina suas previsões para obter um resultado final mais robusto e preciso.
# Uma árvore implementa uma função por partes, do tipo:
# f(x) = {1, se (x1 > t1 e x2 <= t2)
#        0, caso contrário} 
#
# ////////////////////////////////////////////////////////////////
#
# f(x) = {c1, se x pertence a R1
#         c2, se x pertence a R2
#         ...
#         cn, se x pertence a Rn}
# 
# Onde R1, R2, ..., Rn são regiões do espaço de entrada definidas pelas divisões feitas na árvore, e c1, c2, ..., cn são as constantes (decisões) associadas a cada região.
# Cada árvore na floresta faz suas próprias divisões e atribui suas próprias constantes, e o Random Forest combina essas decisões para fazer uma previsão final
# A árvore escolhe as melhores perguntas (divisões) com base em critérios como Gini impurity ou Information Gain, que medem a qualidade das divisões em termos de pureza das classes resultantes.
# O padrão é Gini Impurity
# Definição do Gini em um nó:
# Em um nó temos p0 = proporção de exemplos da classe 0 
# p1 = proporção de exemplos da classe 1
# Gini = 1 - (p0^2 + p1^2)
# Gini = 0 > nó puro (só uma classe) ou todos os exemplos da mesma classe
# Gini alto > nó impuro (exemplos de várias classes)
# O objetivo é minimizar o Gini impurity em cada divisão, ou seja, queremos que as divisões resultem em nós onde a maioria dos exemplos pertença a uma única classe, tornando os nós mais "puros".
# O Gini impurity varia entre 0 (nó perfeitamente puro, onde todos os exemplos pertencem à mesma classe) e 0,5 (nó completamente impuro, onde as classes estão igualmente distribuídas).
# Durante o processo de construção da árvore, o algoritmo avalia todas as possíveis divisões para cada característica e escolhe aquela que resulta na maior redução do Gini impurity.
# Esse processo é repetido recursivamente para cada nó até que um critério de parada seja atingido (por exemplo, profundidade máxima da árvore, número mínimo de exemplos em um nó, etc.).

In [66]:
# Check vtr
df_clima['chove_amanha_vtr'].value_counts()


chove_amanha_vtr
0    44095
1    12475
Name: count, dtype: int64

In [67]:
# cria o modelo Random Forest com alguns parâmetros importantes


rf = RandomForestClassifier(
    n_estimators=300, # quantidade de árvores // quanto mais árvores, mais estável, porém mais lento
    random_state=42, # fixa a aleatoriedade para conseguir repetir o mesmo resultado
    n_jobs=-1, # -1 significa usar todos os núcleos do processador (fica mais rápido)
    #class_weight={0:1, 1:2}  # dá mais peso para a classe "chuva" (1) para lidar com o desbalanceamento
    class_weight='balanced'  # ajusta automaticamente os pesos das classes com base na frequência
)

# cria um pipeline completo chamado modelo_rf
# a ideia do pipeline é: primeiro tratar os dados, depois treinar o modelo
# isso evita você esquecer de aplicar o preprocessamento no teste
modelo_rf = Pipeline(steps=[
    # etapa de preprocessamento
    # aqui entra o ColumnTransformer criado acima
    # tratar colunas numéricas (imputer mediana + scaler)
    # tratar colunas categóricas (imputer mais frequente + one-hot)
    ('preprocessamento', preprocessador),

    # etapa 2: modelo
    # depois de transformar tudo em números, o Random Forest treina e aprende padrões
    ('modelo', rf)
])


In [68]:
# cria o TimeSeriesSplit com 5 divisões
# cada divisão respeita o tempo:
# treina no passado e testa no futuro
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

# cria uma lista vazia para guardar a acurácia de cada split
accuracies_rf = []

# cria uma lista vazia para guardar o recall da classe "chuva" (valor 1) de cada split
recalls_chuva_rf = []

# para cada divisão temporal criada pelo TimeSeriesSplit
# train_idx recebe índices (posições) que serão usados para treino
# test_idx  recebe índices (posições) que serão usados para teste
for train_idx, test_idx in tscv.split(X):

    # X_train recebe as linhas de X que correspondem ao treino (passado)
    X_train = X.iloc[train_idx]

    # X_test recebe as linhas de X que correspondem ao teste (futuro)
    X_test = X.iloc[test_idx]

    # y_train recebe os valores reais (0/1) do treino
    y_train = y.iloc[train_idx]

    # y_test recebe os valores reais (0/1) do teste
    y_test = y.iloc[test_idx]

    # treina o pipeline completo (preprocessamento + Random Forest)
    # ~~ o preprocessador aprende como tratar os dados no treino
    # ~~ o Random Forest aprende padrões para prever chuva
    modelo_rf.fit(X_train, y_train)

    # faz previsões no conjunto de teste
    # y_pred recebe uma lista de 0 e 1, onde:
    # 0 = não chove
    # 1 = chove
    #y_pred = modelo_rf.predict(X_test)
    probabilidades = modelo_rf.predict_proba(X_test)[:,1]
    y_pred = (probabilidades >= 0.3).astype(int)

    # calcula a acurácia do split atual e guarda dentro da lista accuracies_rf
    # acurácia = total de acertos / total de previsões
    accuracies_rf.append(accuracy_score(y_test, y_pred))

    # calcula o recall da chuva (classe 1) do split atual e guarda na lista recalls_chuva_rf
    # recall chuva responde: "das chuvas que realmente aconteceram, quantas eu detectei?"
    recalls_chuva_rf.append(recall_score(y_test, y_pred))

# imprime a média das acurácias dos 5 splits (resultado mais robusto do que 1 split só)
print(f"Acurácia (RF): {np.mean(accuracies_rf):.2f}")

# imprime a média do recall da chuva dos 5 splits (muito importante para não perder dias de chuva)
print(f"Recall - chuva (RF): {np.mean(recalls_chuva_rf):.2f}")


Acurácia (RF): 0.83
Recall - chuva (RF): 0.72


In [ ]:
# Matriz de confusão
# gera a matriz de confusão comparando y_test >>> valores reais e y_pred >>> valores previstos pelo modelo
# o resultado:::
# TN = acertos de "não chove"
# FP = falso alarme de chuva
# FN = disse "não chove", mas choveu
# TP = acertou a chuva
confusion_matrix(y_test, y_pred)

array([[6381,  988],
       [ 629, 1430]])

In [69]:
# imprime a comparação de acurácia média entre os dois modelos
print(f"A Acurácia Reg Logística é de: {np.mean(accuracies):.2f}")
print(f"A Acurácia Random Forest é de: {np.mean(accuracies_rf):.2f}")

# imprime a comparação do recall de chuva (classe 1) entre os dois modelos
print(f"Recall chuva Reg Logística - Quando o modelo prediz chuva: {np.mean(recalls_chuva):.2f} das vezes")
print(f"Recall chuva Random Forest - Quando o modelo prediz chuva: {np.mean(recalls_chuva_rf):.2f} das vezes")

A Acurácia Reg Logística é de: 0.85
A Acurácia Random Forest é de: 0.83
Recall chuva Reg Logística - Quando o modelo prediz chuva: 0.55 das vezes
Recall chuva Random Forest - Quando o modelo prediz chuva: 0.72 das vezes


In [91]:
# Gradient Boosting Classifier
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html
# https://scikit-learn.org/stable/modules/ensemble.html#gradient-boosting
# Treina modelos sequenciais onde cada novo modelo tenta corrigir os erros do modelo anterior.
# Cada árvore é treinada para prever os resíduos (erros) do modelo anterior, e as previsões finais são obtidas somando as previsões de todas as árvores.
# O Gradient Boosting é eficaz para melhorar a precisão do modelo, especialmente em conjuntos de dados complexos, mas pode ser mais sensível ao overfitting se não for adequadamente regulado.
# Definição do modelo Gradient Boosting Classifier com parâmetros importantes
# função dok Gradient Boosting Classifier
# F(x) = F_0(x) + \sum_{m=1}^{M} \gamma_m h_m(x)
# F0(x) é a previsão inicial (por exemplo, a média dos valores alvo)
# hm(x) é a m-ésima árvore de decisão treinada para prever os resíduos do modelo anterior // árvore fraca
# γm é a taxa de aprendizado que controla o impacto de cada árvore na previsão final // peso da árvore
# M é o número total de árvores na sequência
# *** Função de perda (loss function) é usada para medir o quão bem o modelo está se saindo e guiar o processo de aprendizado.
# L(y, \hat{y}) = -\left[y \log(\hat{y}) + (1-y)\log(1-\hat{y})\right]
# Onde y é o valor real (0 ou 1) e ŷ é a probabilidade prevista pelo modelo para a classe positiva (1).
# No caso de classificação binária, uma função de perda comum é a log-loss (ou logistic loss), que penaliza previsões incorretas de forma mais severa.
# atualização do modelo em cada iteração:
# F_m(x) = F_{m-1}(x) + \gamma_m h_m(x)
# Onde Fm(x) é o modelo atualizado após a m-ésima árvore, Fm-1(x) é o modelo anterior, hm(x) é a nova árvore treinada para prever os resíduos, e γm é a taxa de aprendizado.
# A cada iteração, o modelo é ajustado para reduzir os erros cometidos pelo modelo anterior, melhorando assim a precisão geral.
# Definição do modelo Gradient Boosting Classifier com parâmetros importantes
# cria o modelo Gradient Boosting Classifier com alguns parâmetros importantes
# n_estimators = número de árvores na sequência
# learning_rate = taxa de aprendizado que controla o impacto de cada árvore


gbc = GradientBoostingClassifier(
    n_estimators=300, # número de árvores na sequência
    learning_rate=0.5, # taxa de aprendizado que controla o impacto de cada árvore
    max_depth=3, # profundidade máxima de cada árvore
    random_state=42, # fixa a aleatoriedade para resultados reproduzíveis
    #max_leaf_nodes= 50 # limita o número de nós folha em cada árvore para evitar overfitting
)

In [92]:
# var recebe o pipeline do Gradient Boosting
modelo_gb = Pipeline(steps=[('preprocessamento', preprocessador),('modelo', gbc)])

In [93]:
# cria o TimeSeriesSplit com 5 divisões
tscv = TimeSeriesSplit(n_splits=5)

# listas para armazenar métricas
accuracies_gb = []
recalls_chuva_gb = []

# para cada split temporal
for train_idx, test_idx in tscv.split(X):

    # separa treino e teste respeitando o tempo
    X_train = X.iloc[train_idx]
    X_test  = X.iloc[test_idx]
    y_train = y.iloc[train_idx]
    y_test  = y.iloc[test_idx]

    # treina o modelo Gradient Boosting
    modelo_gb.fit(X_train, y_train)

    # gera previsões
    y_pred = modelo_gb.predict(X_test)

    # armazena acurácia
    accuracies_gb.append(accuracy_score(y_test, y_pred))

    # armazena recall da chuva (classe 1)
    recalls_chuva_gb.append(recall_score(y_test, y_pred))
    
# imprime resultados finais do Gradient Boosting
print(f"Acurácia Gradient Boosting: {np.mean(accuracies_gb):.2f}")
print(f"Recall chuva Gradient Boosting - Quando o modelo prediz chuva: {np.mean(recalls_chuva_gb):.2f} das vezes")


Acurácia Gradient Boosting: 0.86
Recall chuva Gradient Boosting - Quando o modelo prediz chuva: 0.54 das vezes


In [90]:
# Matriz de confusão
# gera a matriz de confusão comparando y_test >>> valores reais e y_pred >>> valores previstos pelo modelo
# o resultado:::
# TN = acertos de "não chove"
# FP = falso alarme de chuva
# FN = disse "não chove", mas choveu
# TP = acertou a chuva
confusion_matrix(y_test, y_pred)

array([[6993,  376],
       [1006, 1053]])